In [ ]:
# Bootstrap (CI/offline, deterministic)
import os, random, numpy as np
# headless plotting (some cells call plt.*)
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

# core API we need
from ts2eg.core import growth_payoffs, nmf_on_X, estimate_A_from_series, find_ESS
import ts2eg as gm
try:
    from ts2eg import extensions as ext
except Exception:
    ext = None

# seeds
os.environ.setdefault("TS2EG_CI", "1")
os.environ["PYTHONHASHSEED"] = "0"
random.seed(0); np.random.seed(0)

# tiny offline data stub (only if notebook didn't define data yet)
g = globals()
if "counts" not in g and "v_growth" not in g and "X" not in g:
    rng = np.random.default_rng(0)
    N, T = 4, 80
    counts = np.maximum(rng.lognormal(mean=0.0, sigma=0.4, size=(N, T)), 1e-8)

# default K if unspecified
if "K" not in g:
    K = 3


In [ ]:
# Bootstrap (CI-safe)
# tags: [bootstrap]
import os, random, numpy as np
import ts2eg

SEED = 0
random.seed(SEED)
np.random.seed(SEED)

# Downshift knob: set TS2EG_CI=1 in CI to keep runtime small.
TS2EG_CI = os.getenv("TS2EG_CI") == "1"


# BIO_DEMO: Ecological time series → Evolutionary Game analysis

This notebook demonstrates two biologically interpretable payoff constructions:

1. **Per-capita growth payoffs** `growth_payoffs` (standard Malthusian fitness), suitable for taxa/subclones.
2. **Information-gain payoffs** `info_gain_payoffs` (LOFO ablation for forecasting a target), useful when a scalar endpoint (e.g., biomass, metabolite, clinical score) is available.

We simulate a small Lotka–Volterra (LV) community, learn **strategy archetypes** via NMF, fit a **strategy-level payoff operator** `A`, and search for **ESS** using `find_ESS`.

In [ ]:
# --- bootstrap so `import ts2eg` works before installation ---
import sys, os
for up in [os.getcwd(), os.path.dirname(os.getcwd()), os.path.dirname(os.path.dirname(os.getcwd()))]:
    src = os.path.join(up, "src")
    if os.path.isdir(src) and src not in sys.path:
        sys.path.insert(0, src)
        break
import ts2eg as gm


## 1) Simulate a small LV community (counts)
We generate counts for `N=5` taxa over `T=200` time points with weak competition and noise.

In [ ]:
def simulate_lv(N=5, T=200, dt=0.1, noise_sd=0.02, seed=0):
    rng = np.random.default_rng(seed)
    r = rng.normal(0.2, 0.05, size=N)  # intrinsic growth
    # interaction matrix: diagonally dominant competition
    A = 0.3*np.eye(N) + 0.05*rng.random((N,N))
    x = np.maximum(rng.lognormal(mean=0.0, sigma=0.4, size=N), 1e-3)
    X = np.zeros((N, T))
    for t in range(T):
        X[:, t] = x
        g = r - A @ x
        x = x * np.exp(g*dt + noise_sd*rng.standard_normal(N)*np.sqrt(dt))
        x = np.maximum(x, 1e-8)
    return X

counts = simulate_lv()
N, T = counts.shape
taxa = [f'Taxon_{i+1}' for i in range(N)]
print('counts shape:', counts.shape)
pd.DataFrame(counts, index=taxa).iloc[:, :5]

## 2) Payoffs = per-capita growth (Malthusian fitness)
`v_i(t) = [log n_i(t+1) - log n_i(t)] / dt` padded to length `T`.

In [ ]:
v_growth = growth_payoffs(counts, dt=1.0, pad='edge')
print('v_growth shape:', v_growth.shape)
pd.DataFrame(v_growth, index=taxa).iloc[:, :5]

In [ ]:
# CI: define X explicitly from counts to keep shapes consistent
import numpy as np
X = np.asarray(counts, dtype=float)


## 3) Strategy basis via `nmf_on_X` on **relative abundances**
We factor the relative-abundance matrix `X` into `S @ H` with `S` column-normalized. `K=3` archetypes.
Columns of `S` are interpretable **guild archetypes**; we then infer time-varying memberships `x(t)` inside `estimate_A_from_series`.

In [ ]:
# --- Canonical imports (ts2eg only) ---
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from ts2eg.core import nmf_on_X, growth_payoffs, estimate_A_from_series, find_ESS
import ts2eg as gm
try:
    from ts2eg import extensions as ext
except Exception:
    ext = None  # optional


## 4) Estimate strategy-level operator `A` and find ESS (growth payoffs)

In [ ]:
# Ensure strategy basis S exists before estimation (and ensure X first)
import numpy as _np
if 'X' not in globals():
    if 'v_growth' in globals():
        X = _np.asarray(v_growth, dtype=float)
    elif 'counts' in globals():
        X = _np.asarray(counts, dtype=float)
    else:
        raise NameError("X is undefined; expected v_growth or counts earlier in the notebook.")
try:
    S  # noqa: F821
except NameError:
    try:
        K = int(globals().get('K', 3))
    except Exception:
        K = 3
    S, H = nmf_on_X(X, k=K, iters=50, seed=1, normalize='l2')


In [ ]:
# Ensure strategy basis S exists before estimation
try:
    _ = S  # noqa: F821
except NameError:
    try:
        K = int(globals().get('K', 3))
    except Exception:
        K = 3
    S, H = nmf_on_X(X, k=K, iters=50, seed=1, normalize='l2')


In [ ]:
est = estimate_A_from_series(S, X, v_growth, k=K, ridge=1e-2)
A = est['A']
R2 = est['R2']
print('R^2 (growth payoffs):', round(R2, 3))
ess = [r for r in find_ESS(A, tol=1e-8, max_support=K) if r['is_ess']]
print('ESS count:', len(ess))
for r in ess:
    print('ESS support:', r['support'], 'x*=', np.round(r['x'], 3))

# Decompose A
As = 0.5*(A + A.T)
Aa = 0.5*(A - A.T)
print('||A_s||_F, ||A_a||_F =', float(np.linalg.norm(As)), float(np.linalg.norm(Aa)))

### Plot inferred strategy memberships `x(t)`

In [ ]:
plt.figure(figsize=(8,3))
for i in range(K):
    plt.plot(est['Xk'][i], label=f'x_{i+1}')
plt.title('Strategy memberships over time'); plt.legend(); plt.show()

## 5) Information-gain payoffs for a target (e.g., biomass proxy)
We synthesize a scalar target influenced by two taxa and compute rolling LOFO ablation payoffs.

In [ ]:
rng = np.random.default_rng(3)
target = 0.6*X[0] - 0.4*X[2] + 0.2*rng.standard_normal(T)
v_info = info_gain_payoffs(X, target, ridge=1e-2, window=40)
est_info = estimate_A_from_series(S, X, v_info, k=K, ridge=1e-2)
A_info = est_info['A']
print('R^2 (info-gain payoffs):', round(est_info['R2'], 3))
ess_info = [r for r in find_ESS(A_info, tol=1e-8, max_support=K) if r['is_ess']]
print('ESS count (info):', len(ess_info))

## 6) Null test (circular row shifts)
We break coordination by circularly shifting each taxon independently and repeat the fit.

In [ ]:
def circular_shift_rows(M):
    N, T = M.shape
    rng = np.random.default_rng(9)
    Y = np.zeros_like(M)
    for i in range(N):
        k = int(rng.integers(0, T))
        Y[i] = np.roll(M[i], k)
    return Y

X_null = circular_shift_rows(X)
v_null = growth_payoffs(counts, dt=1.0, pad='edge')  # keep same v here; you could also shift v
S_null, _ = nmf_multiplicative(X_null, r=K, iters=300, seed=5)
S_null = S_null / (np.linalg.norm(S_null, axis=0, keepdims=True) + 1e-12)
est_null = estimate_A_from_series(S_null, X_null, v_null, k=K, ridge=1e-2)
print('R^2 (null):', round(est_null['R2'], 3))
ess_null = [r for r in find_ESS(est_null['A'], tol=1e-8, max_support=K) if r['is_ess']]
print('ESS count (null):', len(ess_null))